In [6]:
# stock_agent.py
import os
import pandas as pd
import yfinance as yf
from prophet import Prophet
from openai import OpenAI

# ========== CONFIG ==========
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Set before running
client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None

# ========== FETCH DATA ==========
def fetch_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    hist = stock.history(period="1y")
    balance_sheet = stock.balance_sheet
    income_stmt = stock.financials

    try:
        net_income = income_stmt.loc["Net Income"].iloc[0]
        total_equity = balance_sheet.loc["Total Stockholder Equity"].iloc[0]
        roe = round(net_income / total_equity, 4) if total_equity != 0 else None
    except:
        roe = None

    trailing_pe = info.get("trailingPE", None)

    summary = {
        "Company Name": info.get("longName", "N/A"),
        "Ticker": ticker,
        "Current Price": info.get("currentPrice", "N/A"),
        "PE Ratio": trailing_pe,
        "ROE": roe,
        "Market Cap": info.get("marketCap", "N/A"),
        "Industry": info.get("industry", "N/A")
    }
    return pd.DataFrame([summary]), hist.tail(5)

# ========== PREPARE PRICE DATA ==========
def get_price_data(ticker):
    df = yf.download(ticker, period="1y")[['Close']].reset_index()
    df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'])
    df.columns = df.columns.droplevel(1)
    return df.dropna()

# ========== GROWTH RATE ==========
def get_growth_rate(ticker):
    info = yf.Ticker(ticker).info
    roe = info.get("returnOnEquity")
    payout = info.get("payoutRatio")
    if roe is None or payout is None:
        return None
    return round(roe * (1 - payout), 2)

# ========== FORECAST ==========
def forecast_price_prophet(df, n_days):
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=n_days)
    forecast = model.predict(future)
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# ========== VALUATION ==========
def simple_dcf(current_price, growth_rate, discount_rate, years=5):
    fcf = current_price * 0.05
    npv = 0
    for t in range(1, years + 1):
        fcf *= (1 + growth_rate)
        npv += fcf / ((1 + discount_rate) ** t)
    terminal_value = fcf * (1 + growth_rate) / (discount_rate - growth_rate)
    npv += terminal_value / ((1 + discount_rate) ** years)
    return round(npv, 2)

def simple_ddm(ticker, growth_rate, discount_rate):
    dividends = yf.Ticker(ticker).dividends.tail(1)
    if dividends.empty:
        return None
    p = dividends.values[0] * (1 + growth_rate) / (discount_rate - growth_rate)
    return round(p, 2)

# ========== ANALYSIS RESULT ==========
def generate_result(forecast, dcf_value, current_price):
    final_price = forecast['yhat'].values[-1]
    trend = "Uptrend" if final_price > current_price else "Downtrend"
    valuation = "Undervalued" if dcf_value > current_price else "Overvalued"
    return trend, valuation

def generate_summary(ticker, trend, dcf_value, ddm_value, valuation, current_price, growth_rate):
    return f"""
🔍 Stock Analysis Report: {ticker}
📈 Forecasted Trend: {trend}
💰 DCF Valuation: {dcf_value} (Current Price: {round(current_price, 2)}, Growth Rate: {growth_rate}) → {valuation}
💰 DDM Valuation: {ddm_value} (Current Price: {round(current_price, 2)}, Growth Rate: {growth_rate}) → {valuation}
🧠 Recommendation: {'Consider Buying' if valuation == 'Undervalued' else 'Cautious Observation'}
"""

# ========== AI INVESTMENT OPINION ==========
def generate_investment_opinion_openai(ticker, trend, valuation):
    if not client:
        return "(⚠️ OpenAI API key not set. Skipping AI opinion.)"
    prompt = f"""
You are a senior investment advisor. Based on the following:
Trend Forecast: {trend}
Valuation Status: {valuation}
Write a professional, concise investment opinion for {ticker} in 2-3 sentences.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a professional equity investment analyst."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"(⚠️ AI generation failed: {e})"

# ========== MAIN PIPELINE ==========
def main():
    ticker = input("Enter stock symbol (e.g., AAPL): ").strip().upper()
    n_days = int(input("Days to predict: ").strip() or 30)

    summary_df, recent_price = fetch_financial_data(ticker)
    print("\n📊 Company Summary:\n", summary_df.T)
    print("\n📈 Last 5 days' prices:\n", recent_price)

    df = get_price_data(ticker)
    forecast = forecast_price_prophet(df, n_days)
    current_price = df['y'].values[-1]
    growth_rate = get_growth_rate(ticker) or 0.05

    dcf_value = simple_dcf(current_price, growth_rate, 0.08)
    ddm_value = simple_ddm(ticker, growth_rate, 0.08)
    trend, valuation = generate_result(forecast, dcf_value, current_price)

    summary = generate_summary(ticker, trend, dcf_value, ddm_value, valuation, current_price, growth_rate)
    print(summary)

    ai_opinion = generate_investment_opinion_openai(ticker, trend, valuation)
    print("🤖 AI Opinion:", ai_opinion)

if __name__ == "__main__":
    main()


Enter stock symbol (e.g., AAPL):  AAPL
Days to predict:  7



📊 Company Summary:
                                   0
Company Name             Apple Inc.
Ticker                         AAPL
Current Price                202.92
PE Ratio                  30.792109
ROE                            None
Market Cap            3011413803008
Industry       Consumer Electronics

📈 Last 5 days' prices:
                                  Open        High         Low       Close  \
Date                                                                        
2025-07-30 00:00:00-04:00  211.899994  212.389999  207.720001  209.050003   
2025-07-31 00:00:00-04:00  208.490005  209.839996  207.160004  207.570007   
2025-08-01 00:00:00-04:00  210.869995  213.580002  201.500000  202.380005   
2025-08-04 00:00:00-04:00  204.509995  207.880005  201.679993  203.350006   
2025-08-05 00:00:00-04:00  203.399994  205.339996  202.160004  202.919998   

                              Volume  Dividends  Stock Splits  
Date                                                          

C:\Users\kkiwi\AppData\Local\Temp\ipykernel_33056\3637761169.py:42: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y")[['Close']].reset_index()
[*********************100%***********************]  1 of 1 completed
16:41:00 - cmdstanpy - INFO - Chain [1] start processing
16:41:01 - cmdstanpy - INFO - Chain [1] done processing



🔍 Stock Analysis Report: AAPL
📈 Forecasted Trend: Uptrend
💰 DCF Valuation: -19.35 (Current Price: 202.92, Growth Rate: 1.27) → Overvalued
💰 DDM Valuation: -0.5 (Current Price: 202.92, Growth Rate: 1.27) → Overvalued
🧠 Recommendation: Cautious Observation

🤖 AI Opinion: While Apple Inc. (AAPL) is currently experiencing an upward trend, its valuation appears to be stretched beyond fundamental support levels. Investors should exercise caution, as the overvalued status may imply limited upside potential in the near term and increased volatility. It may be prudent to consider profit-taking or a wait-and-see approach until valuation metrics align more closely with historical averages.
